# Assignment 4
## Understaning scaling of linear algebra operations on Apache Spark using Apache SystemML

In this assignment we want you to understand how to scale linear algebra operations from a single machine to multiple machines, memory and CPU cores using Apache SystemML. Therefore we want you to understand how to migrate from a numpy program to a SystemML DML program. Don't worry. We will give you a lot of hints. Finally, you won't need this knowledge anyways if you are sticking to Keras only, but once you go beyond that point you'll be happy to see what's going on behind the scenes.

So the first thing we need to ensure is that we are on the latest version of SystemML, which is 1.2.0:

The steps are:
- pip install
- start execution at the cell with the version - check

In [1]:
!pip install systemds==2.2.1

  Using cached systemds-2.2.1-py3-none-any.whl (50.9 MB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)


In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrameReader, SQLContext
from sqlalchemy import create_engine
import pandas as pd
spark= SparkSession.builder.getOrCreate()
spark
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [2]:
import numpy as np
u = np.random.rand(1000,10000)
s = np.random.rand(10000,1000)
w = np.random.rand(1000,1000)

Now we implement a short one-liner to define a very simple linear algebra operation

In case you are unfamiliar with matrxi-matrix multiplication: https://en.wikipedia.org/wiki/Matrix_multiplication

sum(U' * (W . (U * S)))


| Legend        |            |   
| ------------- |-------------| 
| '      | transpose of a matrix | 
| * | matrix-matrix multiplication      |  
| . | scalar multiplication      |   



In [3]:
import time
start = time.time()
res = np.sum(u.T.dot(w * u.dot(s)))
print (time.time()-start)

0.43125224113464355


As you can see this executes perfectly fine. Note that this is even a very efficient execution because numpy uses a C/C++ backend which is known for it's performance. But what happens if U, S or W get such big that the available main memory cannot cope with it? Let's give it a try:

In [5]:
u = np.random.rand(10000,100000)
s = np.random.rand(100000,10000)
w = np.random.rand(10000,10000)

MemoryError: Unable to allocate 7.45 GiB for an array with shape (10000, 100000) and data type float64

After a short while you should see a memory error. This is because the operating system process was not able to allocate enough memory for storing the numpy array on the heap. Now it's time to re-implement the very same operations in SystemDS, and this is your task. Just replace all ###your_code_goes_here sections with proper code, please consider the following table which contains all syntax you need:

| Syntax        |            |   
| ------------- |-------------| 
| M.t()      | transpose of a matrix, where M is the matrix | 
| `M.__matmul__(N)` | matrix-matrix multiplication between M and N      |  
| M * N | scalar multiplication between matrix M and N     |   


## Task

We use SystemDSContext to interface with Apache SystemDS (formerly SystemML). Note that we passed a SparkSession object as parameter to SystemDSContext so now it knows how how to talk to the Apache Spark cluster

In [6]:
from systemds.context import SystemDSContext

with SystemDSContext(spark) as sds:
    # Now we create some large random matrices to have numpy and SystemDS crunch on it
    U = sds.rand(rows=1000,cols=10000)
    S = sds.rand(rows=10000,cols=1000)
    W = sds.rand(rows=1000,cols=1000)
    res = (U.t().__matmul__((W * (U.__matmul__(S))))).sum()
    print(res.compute())

6251512709689.444


In order to show you the advantage of SystemML over numpy we've blown up the sizes of the matrices. Unfortunately, on a 1-2 worker Spark cluster it takes quite some time to complete. Therefore we've stripped down the example to smaller matrices below, but we've kept the code, just in case you are curious to check it out. But you might want to use some more workers which you easily can configure in the environment settings of the project within Watson Studio. Just be aware that you're currently limited to free 50 capacity unit hours per month wich are consumed by the additional workers.

To get consistent results we switch from a random matrix initialization to something deterministic